In [ ]:
# The MIT License (MIT)

# Copyright (c) 2020, NVIDIA CORPORATION.

# Permission is hereby granted, free of charge, to any person obtaining a copy of
# this software and associated documentation files (the "Software"), to deal in
# the Software without restriction, including without limitation the rights to
# use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of
# the Software, and to permit persons to whom the Software is furnished to do so,
# subject to the following conditions:

# The above copyright notice and this permission notice shall be included in all
# copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS
# FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
# COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER
# IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN
# CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE

# Tutorial: Feature Engineering for Recommender Systems

# 0. Initial Data Loading

In our tutorial, we use the [eCommerce behavior data from multi category store](https://www.kaggle.com/mkechinov/ecommerce-behavior-data-from-multi-category-store) from [REES46 Marketing Platform](https://rees46.com/) as our dataset. We define our own goal and filter the dataset accordingly. This jupyter notebook provides the code to preprocess the dataset and generate the train, validation and test sets for the remainder of the tutorial.<br><br>


For our tutorial, we decided that our goal is to predict if a user purchased an item:

<li> Positive: User purchased an item
<li> Negative: User added an item to the cart, but did not purchase it (in the same session)    
<br><br>    
We split the dataset into train, validation and test set by the timestamp:
<li> Training: October 2019 - February 2020
<li> Validation: March 2020
<li> Test: April 2020
<br><br>
We remove AddToCart Events from a session, if in the same session the same item was purchased.

First, download all csv files included in the Google Drive folder.

In [1]:
import pandas as pd
import glob

In [2]:
list_files = glob.glob('../data/*.csv')

Next, we process a single .csv file and extract/filter the rows

In [3]:
def process_files(file):
    df_tmp = pd.read_csv(file)
    df_tmp['session_purchase'] =  df_tmp['user_session'] + '_' + df_tmp['product_id'].astype(str)
    df_purchase = df_tmp[df_tmp['event_type']=='purchase']
    df_cart = df_tmp[df_tmp['event_type']=='cart']
    df_purchase = df_purchase[df_purchase['session_purchase'].isin(df_cart['session_purchase'])]
    df_cart = df_cart[~(df_cart['session_purchase'].isin(df_purchase['session_purchase']))]
    df_cart['target'] = 0
    df_purchase['target'] = 1
    df = pd.concat([df_cart, df_purchase])
    df = df.drop('category_id', axis=1)
    df = df.drop('session_purchase', axis=1)
    df[['cat_0', 'cat_1', 'cat_2', 'cat_3']] = df['category_code'].str.split("\.", n = 3, expand = True).fillna('NA')
    df['brand'] = df['brand'].fillna('NA')
    df = df.drop('category_code', axis=1)
    df['timestamp'] = pd.to_datetime(df['event_time'].str.replace(' UTC', ''))
    df['ts_hour'] = df['timestamp'].dt.hour
    df['ts_minute'] = df['timestamp'].dt.minute
    df['ts_weekday'] = df['timestamp'].dt.weekday
    df['ts_day'] = df['timestamp'].dt.day
    df['ts_month'] = df['timestamp'].dt.month
    df['ts_year'] = df['timestamp'].dt.year
    df.to_csv('./' + file.replace('../data/', ''), index=False)

In [10]:
list_files

['./2019-Dec.csv',
 './2020-Jan.csv',
 './2020-Feb.csv',
 './2019-Oct.csv',
 './2020-Mar.csv',
 './2020-Apr.csv',
 './2019-Nov.csv']

In [4]:
for file in list_files:
    print(file)
    process_files(file)

Finally, we load all preprocessed csv files, combine them and create our train, validation and test sets.

In [13]:
lp = []
list_files = glob.glob('./*.csv')

In [14]:
for file in list_files:
    lp.append(pd.read_csv(file))

/conda/envs/rapids/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [16]:
df = pd.concat(lp)

In [17]:
df.shape

(16695562, 19)

In [21]:
df_test = df[df['ts_month']==4]
df_valid = df[df['ts_month']==3]
df_train = df[(df['ts_month']!=3)&(df['ts_month']!=4)]

In [22]:
df_train.shape, df_valid.shape, df_test.shape

((11461357, 19), (2461719, 19), (2772486, 19))

In [25]:
df_train.to_parquet('../data/train.parquet', index=False)

In [28]:
df_valid.to_parquet('../data/valid.parquet', index=False)

In [27]:
df_test.to_parquet('../data/test.parquet', index=False)